In [ ]:
!pip install prophet

import prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Univariate Forecasting For Heat Wave**

In [1]:
!pip install statsforecast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.8/187.8 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.2/234.2 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 15.4 MB/s eta 0:00:00


**LSTM**

In [2]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler

In [3]:
class LSTM_Model:
  data=None
  timestep=None
  LSTM_Neurons=None
  scaler=None
  trained_model=None

  def __init__(self,LSTM_Neurons,timestep):
     self.LSTM_Neurons=LSTM_Neurons
     self.timestep=timestep
  
  #Loading the data from excel
  def load_data(self,data):
    self.data=np.array(data)

  def get_scaler(self):
    return self.scaler
    
  #Preprocessing and scaling the data
  def preprocess_data(self):
    self.scaler = MinMaxScaler()
    self.scaler = self.scaler.fit(self.data.reshape(-1, 1))
    scaled_data = self.scaler.transform(self.data.reshape(-1, 1))
    return scaled_data

  def create_data_and_labels(self):
    scaled_data=self.preprocess_data()
    X = []
    y = []
    for i in range(self.timestep, len(self.data)):
        X.append(scaled_data[i-self.timestep:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)
    return X,y

  def create_training_and_testing_data(self,size,X,y):
    # Split the data into training and testing sets
    split_idx = int(size * len(X))
    X_train, X_test = X, X[split_idx:]
    y_train, y_test = y, y[split_idx:]
    return X_train,y_train,X_test,y_test
    
  # Define the LSTM model architecture
  def create_LSTM_Model(self):
    model = Sequential()
    model.add(LSTM(self.LSTM_Neurons, input_shape=(self.timestep, 1),activation="tanh"))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

  def fit_data(self,model,X_train,y_train):
    model.fit(X_train, y_train, epochs=100, batch_size=32,verbose=False)
    return model

  def predict(self,model,X_test,y_test):
    y_pred = model.predict(X_test)
    y_pred = self.scaler.inverse_transform(y_pred)
    y_test = self.scaler.inverse_transform(y_test.reshape(1,-1))

    self.y_pred=y_pred
    self.y_test=y_test
    return y_pred,y_test

  def evaluate_rmse(self,model,y_pred,y_test):
    mse = np.mean((y_pred - y_test)**2)
    print("Mean squared error:", mse)

  def plot_test_and_predicted(self,y_pred,y_test):
    fig=plt.figure(figsize=(7,7))
    plt.plot([i[0] for i in y_pred],label='Predicted',color='orange')
    plt.plot(y_test[0],label="Actual",color='blue')
    plt.legend()
    plt.show()
  
  def getter(self):
    return self.y_pred,self.y_test

  def get_model(self):
    return self.trained_model

  def forecast_future(self,forecast_horizon):
    last_values = self.data[-self.timestep:]
    input_sequence = self.scaler.transform(last_values.reshape(-1, 1)).reshape(1, self.timestep)

    # Generate `num_predictions` future values using the model
    output=[]
    for i in range(forecast_horizon):
        # Predict the next value based on the previous `timesteps` values
        next_value = self.trained_model.predict(input_sequence,verbose=False)[0, 0]
        
        # Add the predicted value to the input sequence and remove the first value
        input_sequence = np.concatenate([input_sequence[:, 1:], [[next_value]]], axis=1)
        
        # Scale the input sequence for the next prediction
        input_sequence_scaled = self.scaler.inverse_transform(input_sequence.reshape(-1, 1)).reshape(1, self.timestep, 1)
        
        # Print the predicted value
        output.append(next_value)
    return self.scaler.inverse_transform(np.array(output).reshape(-1,1))
    
  def execute(self,size):
    X,y=self.create_data_and_labels()
    X_train,y_train,X_test,y_test=self.create_training_and_testing_data(size,X,y)
    model=self.create_LSTM_Model()
    fitted_model=self.fit_data(model,X_train,y_train)
    y_pred,y_test=self.predict(fitted_model,X_test,y_test)
    #self.plot_test_and_predicted(y_pred,y_test)
    #self.evaluate_rmse(fitted_model,y_pred,y_test)
    model=self.create_LSTM_Model()
    fitted_model=self.fit_data(model,X,y)
    self.trained_model=fitted_model

In [1]:
!pip install statsforecast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Model Training and Evaluation**

In [11]:
import pandas as pd
import numpy as np
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA,AutoETS,AutoCES,AutoTheta,MSTL,Holt,HoltWinters
from prophet import Prophet
import datetime
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

class TimeSeriesModelSelector:
    
    def __init__(self, data_file,target,ds,city,freq, train_size=0.85):
        self.data_file = data_file
        self.train_size = train_size
        self.target=target
        self.date=ds
        self.city=city
        self.freq=freq
        self.set_horizon()
        self.models = {'ARIMA': AutoARIMA(season_length=12), 'ETS': AutoETS(season_length=12), 'CES':AutoCES(season_length=12) ,'Theta': AutoTheta(season_length=12),'Holt':Holt(season_length=12),'HoltWinters':HoltWinters(season_length=12) ,'MSTL':MSTL(season_length=[12, 36], trend_forecaster=AutoARIMA()),'Prophet': Prophet,'LSTM':LSTM_Model(50,1)}
       
        
        
    def set_horizon(self):
            if self.freq=='M':
              self.horizon=12
            else:
              self.horizon=30

    def warm_start_params(self,m):
    
        res = {}
        for pname in ['k', 'm', 'sigma_obs']:
            if m.mcmc_samples == 0:
                res[pname] = m.params[pname][0][0]
            else:
                res[pname] = np.mean(m.params[pname])
        for pname in ['delta', 'beta']:
            if m.mcmc_samples == 0:
                res[pname] = m.params[pname][0]
            else:
                res[pname] = np.mean(m.params[pname], axis=0)
        return res   

    def load_data(self):
        self.data = pd.read_excel(self.data_file,self.city)
        self.data=self.data[[self.date,self.target]]
        self.train_data = self.data.iloc[:int(self.train_size * len(self.data))]
        self.test_data = self.data.iloc[int(self.train_size * len(self.data)):]
        self.prop=self.data.copy()
        self.prop.rename(columns={self.date:'ds',self.target:'y'},inplace=True)
        self.train_prop = self.prop.iloc[:int(self.train_size * len(self.prop))]
        self.test_prop = self.prop.iloc[int(self.train_size * len(self.prop)):]
    
    def fit_models(self):
        self.model_predictions = {}
        self.future_predictions={}
        for model_name, model_type in self.models.items():
            if model_name == 'ARIMA':
                #print(f"Fitting {model_name}")
                target=self.target
                model_fit = model_type.fit(y=self.train_data[target].values)
                predictions = model_fit.predict(h=len(self.test_data)+self.horizon)['mean']
                self.model_predictions[model_name] = predictions[:len(self.test_data)]
                future_val=predictions[len(self.test_data):]
                if self.freq=='D':
                  date_range=pd.date_range(start=self.test_data[self.date].tail(1).iloc[0,]+datetime.timedelta(days=1), periods=self.horizon,freq=self.freq)
                else:
                  date_range=pd.date_range(start=self.data[self.date].tail(1).iloc[0,]+relativedelta(months=1), periods=12,freq='MS')
                future_df = pd.DataFrame(list(zip(date_range, future_val)),columns =['Date', self.target+'_Predictions'])
                self.future_predictions[model_name]=future_df
                
                
            elif model_name == 'ETS':
                #print(f"Fitting {model_name}")
                target=self.target
                model_fit = model_type.fit(y=self.train_data[target].values)
                predictions = model_fit.predict(h=len(self.test_data)+self.horizon)['mean']
                self.model_predictions[model_name] = predictions[:len(self.test_data)]
                future_val=predictions[len(self.test_data):]
                if self.freq=='D':
                  date_range=pd.date_range(start=self.test_data[self.date].tail(1).iloc[0,]+datetime.timedelta(days=1), periods=self.horizon,freq=self.freq)
                else:
                  date_range=pd.date_range(start=self.data[self.date].tail(1).iloc[0,]+relativedelta(months=1), periods=12,freq='MS')
                future_df = pd.DataFrame(list(zip(date_range, future_val)),columns =['Date', self.target+'_Predictions'])
                self.future_predictions[model_name]=future_df
                   
            elif model_name == 'CES':
                #print(f"Fitting {model_name}")
                target=self.target
                model_fit = model_type.fit(y=self.train_data[target].values)
                predictions = model_fit.predict(h=len(self.test_data)+self.horizon)['mean']
                self.model_predictions[model_name] = predictions[:len(self.test_data)]
                future_val=predictions[len(self.test_data):]
                if self.freq=='D':
                  date_range=pd.date_range(start=self.test_data[self.date].tail(1).iloc[0,]+datetime.timedelta(days=1), periods=self.horizon,freq=self.freq)
                else:
                  date_range=pd.date_range(start=self.data[self.date].tail(1).iloc[0,]+relativedelta(months=1), periods=12,freq='MS')
                future_df = pd.DataFrame(list(zip(date_range, future_val)),columns =['Date', self.target+'_Predictions'])
                self.future_predictions[model_name]=future_df
                
            elif model_name == 'Theta':
                #print(f"Fitting {model_name}")
                target=self.target
                model_fit = model_type.fit(y=self.train_data[target].values)
                predictions = model_fit.predict(h=len(self.test_data)+self.horizon)['mean']
                self.model_predictions[model_name] = predictions[:len(self.test_data)]
                future_val=predictions[len(self.test_data):]
                if self.freq=='D':
                  date_range=pd.date_range(start=self.test_data[self.date].tail(1).iloc[0,]+datetime.timedelta(days=1), periods=self.horizon,freq=self.freq)
                else:
                  date_range=pd.date_range(start=self.data[self.date].tail(1).iloc[0,]+relativedelta(months=1), periods=12,freq='MS')
                future_df = pd.DataFrame(list(zip(date_range, future_val)),columns =['Date', self.target+'_Predictions'])
                self.future_predictions[model_name]=future_df

            elif model_name == 'Holt':
                #print(f"Fitting {model_name}")
                target=self.target
                model_fit = model_type.fit(y=self.train_data[target].values)
                predictions = model_fit.predict(h=len(self.test_data)+self.horizon)['mean']
                self.model_predictions[model_name] = predictions[:len(self.test_data)]
                future_val=predictions[len(self.test_data):]
                if self.freq=='D':
                  date_range=pd.date_range(start=self.test_data[self.date].tail(1).iloc[0,]+datetime.timedelta(days=1), periods=self.horizon,freq=self.freq)
                else:
                  date_range=pd.date_range(start=self.data[self.date].tail(1).iloc[0,]+relativedelta(months=1), periods=12,freq='MS')
                future_df = pd.DataFrame(list(zip(date_range, future_val)),columns =['Date', self.target+'_Predictions'])
                self.future_predictions[model_name]=future_df

            elif model_name == 'HoltWinters':
                #print(f"Fitting {model_name}")
                target=self.target
                model_fit = model_type.fit(y=self.train_data[target].values)
                predictions = model_fit.predict(h=len(self.test_data)+self.horizon)['mean']
                self.model_predictions[model_name] = predictions[:len(self.test_data)]
                future_val=predictions[len(self.test_data):]
                if self.freq=='D':
                  date_range=pd.date_range(start=self.test_data[self.date].tail(1).iloc[0,]+datetime.timedelta(days=1), periods=self.horizon,freq=self.freq)
                else:
                  date_range=pd.date_range(start=self.data[self.date].tail(1).iloc[0,]+relativedelta(months=1), periods=12,freq='MS')
                future_df = pd.DataFrame(list(zip(date_range, future_val)),columns =['Date', self.target+'_Predictions'])
                self.future_predictions[model_name]=future_df

            elif model_name == 'MSTL':
                #print(f"Fitting {model_name}")
                target=self.target
                model_fit = model_type.fit(y=self.train_data[target].values)
                predictions = model_fit.predict(h=len(self.test_data)+self.horizon)['mean']
                self.model_predictions[model_name] = predictions[:len(self.test_data)]
                future_val=predictions[len(self.test_data):]
                if self.freq=='D':
                  date_range=pd.date_range(start=self.test_data[self.date].tail(1).iloc[0,]+datetime.timedelta(days=1), periods=self.horizon,freq=self.freq)
                else:
                  date_range=pd.date_range(start=self.data[self.date].tail(1).iloc[0,]+relativedelta(months=1), periods=12,freq='MS')
                future_df = pd.DataFrame(list(zip(date_range, future_val)),columns =['Date', self.target+'_Predictions'])
                self.future_predictions[model_name]=future_df

            elif model_name == 'Prophet':
                #print(f"Fitting {model_name}")
                model = model_type(changepoint_prior_scale=0.001,
                                    seasonality_prior_scale=0.01,
                                    daily_seasonality=True,
                                    weekly_seasonality=True,
                                    yearly_seasonality=True)
                

                model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
                model.fit(self.train_prop)
                predictions = model.predict(self.test_prop)
                self.model_predictions[model_name] = predictions['yhat'].values
                m2 = Prophet(changepoint_prior_scale=0.001,
                                    seasonality_prior_scale=0.01,
                                    daily_seasonality=True,
                                    weekly_seasonality=True,
                                    yearly_seasonality=True).add_seasonality(name='monthly', period=30.5, fourier_order=5)
                m2.fit(self.prop, init=self.warm_start_params(model))
                future_dates = m2.make_future_dataframe(periods=self.horizon, freq='MS')
                future_pred=m2.predict(future_dates)
                future_pred=future_pred.tail(self.horizon)[['ds','yhat']].rename(columns={'ds':'Date','yhat':self.target+'_Predictions'})
                self.future_predictions[model_name]=future_pred
              
            elif model_name=='LSTM':
                #print("Fitting LSTM")
                model_type.load_data(self.data[self.target].values)
                model_type.execute(self.train_size)
                self.model_predictions[model_name]=model_type.getter()[0]
                if self.freq=='D':
                  date_range=pd.date_range(start=self.test_data[self.date].tail(1).iloc[0,]+datetime.timedelta(days=1), periods=self.horizon,freq=self.freq)
                else:
                  date_range=pd.date_range(start=self.data[self.date].tail(1).iloc[0,]+relativedelta(months=1), periods=12,freq='MS')
                future_val=model_type.forecast_future(self.horizon)
                future_val=future_val.reshape(1,self.horizon).tolist()[0]
                future_df = pd.DataFrame(list(zip(date_range, future_val)),columns =['Date', self.target+'_Predictions'])
                self.future_predictions[model_name]=future_df


                
    
    def evaluate_models(self):
        self.model_rmse = {}
        for model_name, predictions in self.model_predictions.items():
            rmse = np.sqrt(np.mean((predictions - self.test_data[self.target].values)**2))
            self.model_rmse[model_name] = rmse
        self.best_model = min(self.model_rmse, key=self.model_rmse.get)
        self.best_rmse = self.model_rmse[self.best_model]
    
    def run(self):
        self.load_data()
        self.fit_models()
        self.evaluate_models()
        print("RMSE for different Models")
        print(self.model_rmse)
        
        print(f"The best model for this time series data is {self.best_model} with RMSE of {self.best_rmse}")
        res=self.future_predictions[self.best_model]
        res[self.target+'_Predictions']=np.round(res[self.target+'_Predictions'])
        return res


**City Wise Temperature and AQI Prediction**

In [12]:
def get_citywise_temperature():
  city_list=['Adilabad','Warangal','Karimnagar','Khammam','Nizamabad']
  temperature_predictions={}
  for i in city_list:
    print(f"-----------Training Temperature forecasting Model for {i}-----------")
    ts=TimeSeriesModelSelector('/content/Heat Wave Dataset Cleaned.xlsx','Max Temp (°C)','Date',i,'D')
    val=ts.run()
    temperature_predictions[i]=val
    print("\n")
  return temperature_predictions

models=[]
def get_citywise_aqi():
  city_list=['Adilabad','Warangal','Karimnagar','Khammam','Nizamabad']
  aqi_predictions={}
  for i in city_list:
    
    print(f"-----------Training AQI forecasting Model for {i}-----------")
    ts=TimeSeriesModelSelector('/content/AAQ-AQI-interpolated.xlsx','AQI','DATE',i,'M')
    val=ts.run()
    aqi_predictions[i]=val
    models.append(ts)
    print("\n")
  return aqi_predictions



In [13]:
aqi=get_citywise_aqi()

-----------Training AQI forecasting Model for Adilabad-----------


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/hj9rqda2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/ipic_jki.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=63213', 'data', 'file=/tmp/tmp_u8h7l1i/hj9rqda2.json', 'init=/tmp/tmp_u8h7l1i/ipic_jki.json', 'output', 'file=/tmp/tmp_u8h7l1i/prophet_modelbn5xegev/prophet_model-20230303194540.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
19:45:40 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:45:41 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/n5da1zlr.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/n9gmggks.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

1/1 [==============================] - 1s 822ms/step
RMSE for different Models
{'ARIMA': 7.131759330275221, 'ETS': 5.5907559330375, 'CES': 5.125107933340319, 'Theta': 5.7403979635561, 'Holt': 4.922648910148252, 'HoltWinters': 6.791373396050409, 'MSTL': 24.990272155011237, 'Prophet': 11.20047206823306, 'LSTM': 6.39601452807301}
The best model for this time series data is Holt with RMSE of 4.922648910148252


-----------Training AQI forecasting Model for Warangal-----------


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/00__39vl.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/1l5bobxp.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=40974', 'data', 'file=/tmp/tmp_u8h7l1i/00__39vl.json', 'init=/tmp/tmp_u8h7l1i/1l5bobxp.json', 'output', 'file=/tmp/tmp_u8h7l1i/prophet_modelwec_q928/prophet_model-20230303194558.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
19:45:58 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:46:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/cznjdc6e.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/khll3n9p.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

1/1 [==============================] - 1s 532ms/step
RMSE for different Models
{'ARIMA': 18.583595009227988, 'ETS': 15.62985935201934, 'CES': 17.47203921770442, 'Theta': 15.73966880217755, 'Holt': 16.194405890176082, 'HoltWinters': 13.493517415684732, 'MSTL': 27.60258145458844, 'Prophet': 15.146801361073173, 'LSTM': 15.947239002489425}
The best model for this time series data is HoltWinters with RMSE of 13.493517415684732


-----------Training AQI forecasting Model for Karimnagar-----------


...

1/1 [==============================] - 1s 866ms/step
RMSE for different Models
{'ARIMA': 16.622081656646913, 'ETS': 16.34527403686996, 'CES': 13.157632596933286, 'Theta': 16.861827864703432, 'Holt': 16.52943245411315, 'HoltWinters': 13.628256823458067, 'MSTL': 55.36376187326526, 'Prophet': 34.24553124543938, 'LSTM': 17.988678209746144}
The best model for this time series data is CES with RMSE of 13.157632596933286


-----------Training AQI forecasting Model for Khammam-----------


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/lpv8qvn7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/l13_2vmw.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=12221', 'data', 'file=/tmp/tmp_u8h7l1i/lpv8qvn7.json', 'init=/tmp/tmp_u8h7l1i/l13_2vmw.json', 'output', 'file=/tmp/tmp_u8h7l1i/prophet_modeli53q6ajs/prophet_model-20230303194638.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
19:46:38 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:46:39 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/jbhp_pyt.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/jz_ihaoy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

1/1 [==============================] - 1s 514ms/step
RMSE for different Models
{'ARIMA': 21.00084389781996, 'ETS': 21.241974397459956, 'CES': 15.384691661583085, 'Theta': 21.11566308856642, 'Holt': 21.01350789438791, 'HoltWinters': 17.04469623392396, 'MSTL': 22.411245565953717, 'Prophet': 20.111178387221397, 'LSTM': 20.94655987907759}
The best model for this time series data is CES with RMSE of 15.384691661583085


-----------Training AQI forecasting Model for Nizamabad-----------


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/h3jj3ddo.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/280y9zz2.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=5423', 'data', 'file=/tmp/tmp_u8h7l1i/h3jj3ddo.json', 'init=/tmp/tmp_u8h7l1i/280y9zz2.json', 'output', 'file=/tmp/tmp_u8h7l1i/prophet_model67lv42qg/prophet_model-20230303194656.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
19:46:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:46:57 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/jper696l.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/gkvyctwe.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

1/1 [==============================] - 1s 512ms/step
RMSE for different Models
{'ARIMA': 4.388951515855729, 'ETS': 4.604333456105276, 'CES': 6.290046725800457, 'Theta': 4.379108887108288, 'Holt': 4.3562517552860855, 'HoltWinters': 6.756954939631099, 'MSTL': 12.031869685338249, 'Prophet': 4.240183023556744, 'LSTM': 5.376149397922439}
The best model for this time series data is Prophet with RMSE of 4.240183023556744




In [14]:
temp=get_citywise_temperature()

-----------Training Temperature forecasting Model for Adilabad-----------


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/eibx_68w.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/8cz6pbqn.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=67070', 'data', 'file=/tmp/tmp_u8h7l1i/eibx_68w.json', 'init=/tmp/tmp_u8h7l1i/8cz6pbqn.json', 'output', 'file=/tmp/tmp_u8h7l1i/prophet_modelra9_nu2p/prophet_model-20230303194902.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:49:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:49:03 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/caxj5l8x.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/fdgdkuov.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

9/9 [==============================] - 1s 4ms/step
RMSE for different Models
{'ARIMA': 10.009650072352132, 'ETS': 13.025014943929932, 'CES': 16.400740287554406, 'Theta': 13.131351124453653, 'Holt': 13.660411122847668, 'HoltWinters': 13.400023310024327, 'MSTL': 9.07051982023415, 'Prophet': 2.9447418365813527, 'LSTM': 6.647932184018445}
The best model for this time series data is Prophet with RMSE of 2.9447418365813527


-----------Training Temperature forecasting Model for Warangal-----------


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/62nas4jv.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/4r1f3k8q.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=77296', 'data', 'file=/tmp/tmp_u8h7l1i/62nas4jv.json', 'init=/tmp/tmp_u8h7l1i/4r1f3k8q.json', 'output', 'file=/tmp/tmp_u8h7l1i/prophet_modelkoiklhm6/prophet_model-20230303195134.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:51:34 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:51:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/iysj18bd.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/p1yecuoa.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

9/9 [==============================] - 1s 3ms/step
RMSE for different Models
{'ARIMA': 6.778603169470635, 'ETS': 7.199976846440194, 'CES': 10.11359979594025, 'Theta': 7.282289481389578, 'Holt': 7.492878577880823, 'HoltWinters': 7.2359966040236685, 'MSTL': 6.981292996680551, 'Prophet': 2.6939086208788807, 'LSTM': 5.549714247926918}
The best model for this time series data is Prophet with RMSE of 2.6939086208788807


-----------Training Temperature forecasting Model for Karimnagar-----------


...

9/9 [==============================] - 1s 3ms/step
RMSE for different Models
{'ARIMA': 8.179584395356466, 'ETS': 9.54665853134294, 'CES': 12.489391767959418, 'Theta': 9.517749144137145, 'Holt': 10.062618056417795, 'HoltWinters': 9.775953069134976, 'MSTL': 7.629531353982413, 'Prophet': 3.0565823046421023, 'LSTM': 5.817576448242677}
The best model for this time series data is Prophet with RMSE of 3.0565823046421023


-----------Training Temperature forecasting Model for Khammam-----------


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/335zxqrj.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/j9lraatz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=67986', 'data', 'file=/tmp/tmp_u8h7l1i/335zxqrj.json', 'init=/tmp/tmp_u8h7l1i/j9lraatz.json', 'output', 'file=/tmp/tmp_u8h7l1i/prophet_modelq8tbzlak/prophet_model-20230303195617.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:56:17 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:56:17 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/nr3mcp5a.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/gnf58dt6.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

9/9 [==============================] - 1s 3ms/step
RMSE for different Models
{'ARIMA': 5.73850852336553, 'ETS': 5.92363293632053, 'CES': 8.382906686413772, 'Theta': 5.964473747763968, 'Holt': 6.05914134971433, 'HoltWinters': 5.9372996184103, 'MSTL': 5.8972034204799435, 'Prophet': 2.3866855600374244, 'LSTM': 4.763912207442593}
The best model for this time series data is Prophet with RMSE of 2.3866855600374244


-----------Training Temperature forecasting Model for Nizamabad-----------


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/5xfirjd6.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/x_8c4lxo.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=72909', 'data', 'file=/tmp/tmp_u8h7l1i/5xfirjd6.json', 'init=/tmp/tmp_u8h7l1i/x_8c4lxo.json', 'output', 'file=/tmp/tmp_u8h7l1i/prophet_modelkbwhitdp/prophet_model-20230303195823.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:58:23 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:58:23 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/4e9fp6g7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_u8h7l1i/ju12jeb9.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

9/9 [==============================] - 1s 3ms/step
RMSE for different Models
{'ARIMA': 7.176131871106807, 'ETS': 8.539916863668928, 'CES': 11.47021826838343, 'Theta': 8.605008543684693, 'Holt': 8.917681973648362, 'HoltWinters': 18.61066327582224, 'MSTL': 6.652340723521202, 'Prophet': 2.993165793177583, 'LSTM': 5.88734289306173}
The best model for this time series data is Prophet with RMSE of 2.993165793177583




In [13]:
city_list=['Adilabad','Warangal','Karimnagar','Khammam','Nizamabad']

In [ ]:
for i,j in zip(models,city_list):
  y_pred=i.model_predictions[i.best_model]
  y_test=i.test_data[i.target].values
  date_range=pd.date_range(start=i.train_data[i.date].tail(1).iloc[0,]+relativedelta(months=1), periods=len(i.test_data),freq='MS')
  test_df = pd.DataFrame(list(zip(date_range,y_pred, y_test)),columns =['Date','y_pred', 'y_test'])
  future_pred=i.future_predictions[i.best_model]
  with pd.ExcelWriter('/content/Evaluation.xlsx',mode='a') as writer:
    test_df.to_excel(writer, sheet_name=f"{j}_Test",index=False,header=True)
    future_pred.to_excel(writer, sheet_name=f"{j}_Future_pred",index=False,header=True)
